# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

When solving this task, we expect you'll face (and successfully deal with) some problems or make up the ideas of the model improvement. Some of them are: 

- solving a problem of n-grams frequencies storing for a large corpus;
- taking into account keyboard layout and associated misspellings;
- efficiency improvement to make the solution faster;
- ...

Please don't forget to describe such cases, and what you decided to do with them, in the Justification section.

##### IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

In [1]:
import math
from collections import defaultdict

def preprocess_text(text):
    """
    Convert the input text to lowercase and split it into tokens.

    Parameters:
        text (str): The input string.

    Returns:
        List[str]: A list of lowercase tokens.
    """
    return text.lower().split()

def get_ngrams(corpus, n, smoothing_factor=1):
    """
    Compute the frequency of n-grams from a corpus of texts using simple Laplace smoothing.

    Parameters:
        corpus (List[str]): A list of text lines.
        n (int): The n-gram order (e.g., 1 for unigrams, 2 for bigrams).
        smoothing_factor (int, optional): Smoothing factor added to each observed n-gram count. Default is 1.

    Returns:
        tuple:
            - ngrams (defaultdict): A dictionary with n-gram tuples as keys and their (smoothed) frequency as values.
            - total_ngrams (int): Total number of n-grams in the corpus.
    """
    ngrams = defaultdict(int)
    total_ngrams = 0

    for line in corpus:
        # Add start and end tokens to the sentence
        tokens = ["<s>"] + preprocess_text(line) + ["</s>"]
        for i in range(len(tokens) - n + 1):
            ngram = tuple(tokens[i:i+n])
            ngrams[ngram] += 1
            total_ngrams += 1

    # Apply Laplace smoothing to observed n-grams
    for ngram in ngrams:
        ngrams[ngram] += smoothing_factor

    return ngrams, total_ngrams

def edits1(word):
    """
    Generate a set of words that are one edit away from the input word.
    The possible edits include insertion, deletion, substitution, and transposition of characters.

    Parameters:
        word (str): The original word.

    Returns:
        Set[str]: A set of candidate words that are one edit distance away.
    """
    letters = 'abcdefghijklmnopqrstuvwxyz'
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes = [L + R[1:] for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
    replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
    inserts = [L + c + R for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def get_candidates(word, vocabulary, keyboard_layout=None):
    """
    Generate candidate corrections for a given word.

    - If the word is already present in the vocabulary, it is considered correct.
    - Otherwise, candidates with edit distance 1 are generated, and if none are found,
      candidates with edit distance 2 are considered.
    - If a keyboard layout is provided, candidates that account for common keyboard errors are also included.

    Parameters:
        word (str): The word to be corrected.
        vocabulary (set): A set of known (valid) words.
        keyboard_layout (List[str], optional): List of keyboard rows (e.g., ["qwertyuiop"]) to simulate adjacent key errors. Default is None.

    Returns:
        Set[str]: A set of candidate corrections.
    """
    candidates = set()

    # If the word exists in the vocabulary, it is already correct.
    if word in vocabulary:
        candidates.add(word)
    else:
        # Generate candidates with edit distance 1.
        candidates = edits1(word) & vocabulary
        # If no candidates found at distance 1, try edit distance 2.
        if not candidates:
            candidates = {e2 for e1 in edits1(word) for e2 in edits1(e1) if e2 in vocabulary}

    # Incorporate keyboard layout errors if provided.
    if keyboard_layout:
        kb_candidates = handle_keyboard_errors(word, keyboard_layout)
        candidates |= (kb_candidates & vocabulary)

    # If no candidate is found, return the original word.
    return candidates if candidates else {word}

def handle_keyboard_errors(word, keyboard_layout):
    """
    Handle typical keyboard layout errors by considering adjacent keys.

    Parameters:
        word (str): The input word that might contain a keyboard error.
        keyboard_layout (List[str]): A list of strings where each string represents a row on the keyboard (e.g., "qwertyuiop").

    Returns:
        Set[str]: A set of candidate words generated by substituting a character with its neighbor.
    """
    candidates = set()
    # Build a mapping from character to its row and index in that row.
    layout_dict = {}
    for row in keyboard_layout:
        for i, char in enumerate(row):
            layout_dict[char] = (row, i)

    for i in range(len(word)):
        if word[i] in layout_dict:
            row, col = layout_dict[word[i]]
            # Try replacing the character with its immediate neighbor on the left or right.
            for delta in [-1, 1]:
                new_col = col + delta
                if 0 <= new_col < len(row):
                    candidate = word[:i] + row[new_col] + word[i+1:]
                    candidates.add(candidate)
    return candidates

def probability_of_correction(candidate, context, unigrams, bigrams, trigrams, unigram_total, smoothing_factor=1, weights=(0.1, 0.3, 0.6)):
    """
    Calculate the weighted log probability of a candidate word given its context using interpolated n-gram models.

    The overall score is a weighted sum of the log probabilities from:
      - Unigram model
      - Bigram model (if previous context exists)
      - Trigram model (if two preceding words exist)

    Parameters:
        candidate (str): The candidate word.
        context (List[str]): The list of preceding words (context).
        unigrams (dict): Unigram frequency dictionary.
        bigrams (dict): Bigram frequency dictionary.
        trigrams (dict): Trigram frequency dictionary.
        unigram_total (int): Total count of unigrams.
        smoothing_factor (int, optional): Smoothing factor for Laplace smoothing. Default is 1.
        weights (tuple, optional): Weights for (unigram, bigram, trigram) models. Default is (0.1, 0.3, 0.6).

    Returns:
        float: The interpolated log probability score for the candidate.
    """
    # Calculate unigram probability
    unigram_prob = (unigrams.get((candidate,), 0) + smoothing_factor) / (unigram_total + smoothing_factor * len(unigrams))
    score = weights[0] * math.log(unigram_prob)

    # Calculate bigram probability if previous context exists
    if len(context) >= 1:
        bigram_prob = (bigrams.get((context[-1], candidate), 0) + smoothing_factor) / (unigrams.get((context[-1],), 0) + smoothing_factor * len(unigrams))
        score += weights[1] * math.log(bigram_prob)

    # Calculate trigram probability if two preceding words exist
    if len(context) >= 2:
        trigram_prob = (trigrams.get((context[-2], context[-1], candidate), 0) + smoothing_factor) / (bigrams.get((context[-2], context[-1]), 0) + smoothing_factor * len(bigrams))
        score += weights[2] * math.log(trigram_prob)

    return score

def correct_spelling(text, unigrams, bigrams, trigrams, vocabulary, unigram_total, keyboard_layout=None, smoothing_factor=1, weights=(0.1, 0.3, 0.6)):
    """
    Corrects spelling errors in a given text using context-sensitive n-gram language models and candidate generation.

    For each word in the input text, the function generates candidate corrections and selects the one with the highest
    interpolated log probability score based on unigram, bigram, and trigram models.

    Parameters:
        text (str): The input text to be corrected.
        unigrams (dict): Unigram frequency dictionary.
        bigrams (dict): Bigram frequency dictionary.
        trigrams (dict): Trigram frequency dictionary.
        vocabulary (set): Set of known valid words.
        unigram_total (int): Total count of unigrams.
        keyboard_layout (List[str], optional): List of keyboard rows (e.g., ["qwertyuiop"]). Default is None.
        smoothing_factor (int, optional): Smoothing factor for Laplace smoothing. Default is 1.
        weights (tuple, optional): Weights for (unigram, bigram, trigram) probabilities. Default is (0.1, 0.3, 0.6).

    Returns:
        str: The corrected text.
    """
    words_in_text = preprocess_text(text)
    corrected_words = []
    context = []  # Holds the corrected words as context for subsequent words

    for word in words_in_text:
        # Generate candidate corrections for the current word
        candidates = get_candidates(word, vocabulary, keyboard_layout)
        # Select the candidate with the highest interpolated probability score
        best_candidate = max(candidates, key=lambda c: probability_of_correction(
            c, context, unigrams, bigrams, trigrams, unigram_total, smoothing_factor, weights))
        corrected_words.append(best_candidate)
        context.append(best_candidate)

    return ' '.join(corrected_words)


## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

## 1. Dataset and N-gram Model Building

- **Using big.txt:**  
  I started by using Norvig’s `big.txt` dataset because it’s a well-known, large corpus that provides a wide range of words and word sequences. Building the unigram, bigram, and trigram models on `big.txt` helped me get reliable frequency counts and a solid language model foundation.

- **Testing with wiki_sentences_v2.csv:**  
  After constructing the language model using `big.txt`, I decided to test my corrector on real-world data from Wikipedia using the `wiki_sentences_v2.csv` file. This dataset contains actual sentences, making it a great benchmark for evaluating the performance of my spelling corrector on noisy, realistic text.
  You can find the dataset of Wiki Sentences [here](https://www.kaggle.com/datasets/ved1104/wiki-sentences/data).

## 2. Smoothing Technique

I used simple Laplace smoothing (adding a constant, typically 1) when calculating n-gram frequencies. Although this is a basic method, it was enough for handling unseen n-grams in my experiments.

## 3. Candidate Generation Strategy

- **Edit Distance Candidates:**  
  My approach for candidate generation is inspired by Norvig’s method. I first check if the word is already in my vocabulary. If it isn’t, I generate candidates using edit distance 1, and if necessary, expand to edit distance 2. This method is simple and strikes a good balance between efficiency and quality.

- **Handling Keyboard Errors:**  
  Since many typos occur due to pressing adjacent keys on the keyboard, I added a module to handle keyboard layout errors (using a QWERTY layout). This helps improve correction accuracy by catching common typing mistakes.

## 4. Interpolation Weights for N-gram Models

I assigned weights of **(0.1, 0.3, 0.6)** to the unigram, bigram, and trigram probabilities respectively:

- **Trigram (0.6):**  
  I believe the two-word context from the trigram model provides the most valuable information, so it gets the highest weight.

- **Bigram (0.3):**  
  The immediate previous word is also important, though it’s a bit less informative than the full trigram context.

- **Unigram (0.1):**  
  Overall word frequency is considered, but it plays a smaller role compared to contextual models.

I chose these weights based on my initial experiments and intuition, and I might adjust them later with more thorough testing.



## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity (or just take another dataset). Compare your solution to the Norvig's corrector, and report the accuracies.

In [ ]:
import random
import pandas as pd

def add_noise_to_word(word, noise_prob=0.2):
    """
    With probability `noise_prob`, perturb the word by performing one of the following operations:
      - Deletion: Remove a random character.
      - Insertion: Insert a random character at a random position.
      - Replacement: Replace a random character with another random character.
      - Transposition: Swap two adjacent characters.
      
    If no noise is added (based on the probability), returns the original word.

    Parameters:
        word (str): The original word.
        noise_prob (float): The probability with which to add noise. Default is 0.2.

    Returns:
        str: The potentially noised word.
    """
    # With probability (1 - noise_prob) return the original word
    if random.random() > noise_prob:
        return word

    letters = 'abcdefghijklmnopqrstuvwxyz'
    mod_type = random.choice(['delete', 'insert', 'replace', 'transpose'])
    
    if mod_type == 'delete' and len(word) > 1:
        idx = random.randint(0, len(word) - 1)
        return word[:idx] + word[idx+1:]
    elif mod_type == 'insert':
        idx = random.randint(0, len(word))
        letter = random.choice(letters)
        return word[:idx] + letter + word[idx:]
    elif mod_type == 'replace' and len(word) > 0:
        idx = random.randint(0, len(word) - 1)
        letter = random.choice(letters)
        return word[:idx] + letter + word[idx+1:]
    elif mod_type == 'transpose' and len(word) > 1:
        idx = random.randint(0, len(word) - 2)
        return word[:idx] + word[idx+1] + word[idx] + word[idx+2:]
    
    return word

def add_noise_to_sentence(sentence, noise_prob=0.2):
    """
    Apply noise to each word in the sentence using the add_noise_to_word function.

    Parameters:
        sentence (str): The input sentence.
        noise_prob (float): The probability of applying noise to each word. Default is 0.2.

    Returns:
        str: The sentence after noise has been applied to its words.
    """
    words_in_sentence = sentence.split()
    noisy_words = [add_noise_to_word(word, noise_prob) for word in words_in_sentence]
    return ' '.join(noisy_words)

# ----------------------
# Load and Prepare the Dataset
# ----------------------
# Load the dataset from a CSV file. The CSV should contain a column named 'sentence'.
df = pd.read_csv('wiki_sentences_v2.csv')

# Create a new column with "noisy" sentences.
# Each word in the original sentence is perturbed with a 20% probability.
df['noisy_sentence'] = df['sentence'].apply(lambda s: add_noise_to_sentence(s, noise_prob=0.2))

# ----------------------
# Build Corpus and Vocabulary
# ----------------------
# Extract the corpus from the original sentences
corpus = df['sentence'].tolist()

# Build the vocabulary by splitting each sentence into words and converting them to lowercase
vocabulary = set()
for sentence in corpus:
    vocabulary.update(sentence.lower().split())

# ----------------------
# Build N-gram Models
# ----------------------
# Build unigram, bigram, and trigram frequency models from the corpus.
# The function `get_ngrams` is assumed to be defined elsewhere.
unigrams, unigram_total = get_ngrams(corpus, 1)
bigrams, _ = get_ngrams(corpus, 2)
trigrams, _ = get_ngrams(corpus, 3)

# ----------------------
# Evaluate the Context-Sensitive Spelling Corrector
# ----------------------
total_sentences = len(df)
word_correct = 0   # Counter for correctly corrected words
total_words = 0    # Total number of words processed
sentence_correct = 0  # Counter for sentences that are exactly corrected
corrected_sentences = []  # To store the corrected sentences

# Define a QWERTY keyboard layout (lowercase letters only) to account for keyboard errors.
keyboard_layout = ["qwertyuiop", "asdfghjkl", "zxcvbnm"]

# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    original = row['sentence']
    noisy = row['noisy_sentence']
    
    # Use the context-sensitive correct_spelling function to correct the noisy sentence.
    corrected = correct_spelling(noisy, unigrams, bigrams, trigrams, vocabulary, unigram_total, keyboard_layout)
    corrected_sentences.append(corrected)
    
    # Evaluate at the word level:
    orig_words = original.lower().split()
    corr_words = corrected.lower().split()
    # Compare corresponding words (up to the length of the shorter sentence)
    for o, c in zip(orig_words, corr_words):
        if o == c:
            word_correct += 1
    total_words += len(orig_words)
    
    # Evaluate at the sentence level:
    if original.lower() == corrected.lower():
        sentence_correct += 1

# Save the corrected sentences to a new column in the DataFrame.
df['corrected_sentence'] = corrected_sentences

# Print the evaluation results:
print("Word-level accuracy: {:.2f}%".format((word_correct / total_words) * 100))
print("Sentence-level accuracy: {:.2f}%".format((sentence_correct / total_sentences) * 100))


Word-level accuracy: 97.50%
Sentence-level accuracy: 70.52%


In [4]:
import re
import collections

# ------------------------------
# Norvig's Spelling Corrector Setup
# ------------------------------

def words(text):
    """
    Tokenize the input text by extracting all word characters in lowercase.
    
    Parameters:
        text (str): The text to be tokenized.
    
    Returns:
        List[str]: A list of lowercase word tokens.
    """
    return re.findall(r'\w+', text.lower())

# Build the WORDS frequency dictionary from the original sentences in the dataset.
# This dictionary will serve as the basis for word probabilities.
WORDS = collections.Counter()
for sentence in df['sentence']:
    WORDS.update(words(sentence))
N = sum(WORDS.values())

def P(word, N=N):
    """
    Calculate the probability of a word based on its frequency in the WORDS dictionary.
    
    Parameters:
        word (str): The word for which to calculate the probability.
        N (int): Total count of all words.
    
    Returns:
        float: The probability of the word.
    """
    return WORDS[word] / N

def correction(word):
    """
    Return the most probable correction for the given word.
    
    Parameters:
        word (str): The word to be corrected.
    
    Returns:
        str: The most likely corrected word.
    """
    return max(candidates(word), key=P)

def candidates(word):
    """
    Generate candidate corrections for the input word.
    The candidate set includes:
      - The word itself (if it is known),
      - All words that are one edit distance away,
      - All words that are two edit distances away.
    
    Parameters:
        word (str): The word for which to generate candidate corrections.
    
    Returns:
        set: A set of candidate words.
    """
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words_list):
    """
    Filter a list of words and return only those that exist in the WORDS dictionary.
    
    Parameters:
        words_list (Iterable[str]): A list or iterable of words.
    
    Returns:
        set: A set of words that are present in WORDS.
    """
    return set(w for w in words_list if w in WORDS)

def edits1(word):
    """
    Generate all words that are one edit away from the input word.
    The edits include deletion, transposition, replacement, and insertion.
    
    Parameters:
        word (str): The original word.
    
    Returns:
        set: A set of words one edit distance away.
    """
    letters = 'abcdefghijklmnopqrstuvwxyz'
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:]   for L, R in splits if len(R) > 1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word):
    """
    Generate all words that are two edits away from the input word.
    
    Parameters:
        word (str): The original word.
    
    Returns:
        generator: A generator yielding words two edits away.
    """
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

def norvig_correct_spelling(sentence):
    """
    Correct the spelling of each word in a sentence using Norvig's approach.
    Each word is lowercased, and the correction is performed independently.
    
    Parameters:
        sentence (str): The sentence to be corrected.
    
    Returns:
        str: The corrected sentence.
    """
    tokens = sentence.split()
    corrected_tokens = [correction(token.lower()) for token in tokens]
    return " ".join(corrected_tokens)

# ------------------------------
# Evaluation of Norvig's Corrector on the Dataset
# ------------------------------

# Initialize counters for evaluation metrics.
total_sentences = len(df)
word_correct_norvig = 0         # Count of correctly corrected words.
total_words_norvig = 0          # Total number of words evaluated.
sentence_correct_norvig = 0     # Count of sentences that match exactly.
norvig_corrected_sentences = [] # List to store the corrected sentences.

# Iterate over each row in the dataset.
for index, row in df.iterrows():
    original = row['sentence']
    noisy = row['noisy_sentence']
    
    # Correct the noisy sentence using Norvig's spelling corrector.
    corrected = norvig_correct_spelling(noisy)
    norvig_corrected_sentences.append(corrected)
    
    # Evaluate word-level accuracy by comparing each corresponding word.
    orig_words = original.lower().split()
    corr_words = corrected.lower().split()
    for o, c in zip(orig_words, corr_words):
        if o == c:
            word_correct_norvig += 1
    total_words_norvig += len(orig_words)
    
    # Evaluate sentence-level accuracy (exact match).
    if original.lower() == corrected.lower():
        sentence_correct_norvig += 1

# Add the Norvig corrected sentences to the DataFrame for further analysis.
df['norvig_corrected'] = norvig_corrected_sentences

# Print the evaluation results.
print("Norvig - Word-level accuracy: {:.2f}%".format((word_correct_norvig / total_words_norvig) * 100))
print("Norvig - Sentence-level accuracy: {:.2f}%".format((sentence_correct_norvig / total_sentences) * 100))


Norvig - Word-level accuracy: 78.67%
Norvig - Sentence-level accuracy: 2.22%


#### Useful resources (also included in the archive in moodle):

1. [Possible dataset with N-grams](https://www.ngrams.info/download_coca.asp)
2. [Damerau–Levenshtein distance](https://en.wikipedia.org/wiki/Damerau–Levenshtein_distance#:~:text=Informally%2C%20the%20Damerau–Levenshtein%20distance,one%20word%20into%20the%20other.)